In [1]:
import sys

sys.path.append('../src')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
import torch

import os
from glob import glob

import pickle
import yaml
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from utils.result_visualization import *
from utils.scalers import *
from dataset.sea_fog_dataset import Temporal_Graph_Signal

In [49]:
hour = 6

In [50]:
if hour == 1:
    input_length = 120
    output_length = 6
    
elif hour == 3:
    input_length = 120
    output_length = 18
    
elif hour == 6:
    input_length = 102
    output_length = 36

In [51]:
dirs = glob(f'../exp/1212/IC_PN_BEATS/forecast_{hour}hour*/*')
dirs.sort()
dirs

['../exp/1212/IC_PN_BEATS/forecast_6hour_regression_all_combine_loss/1212_183337',
 '../exp/1212/IC_PN_BEATS/forecast_6hour_regression_all_single_loss/1212_200752',
 '../exp/1212/IC_PN_BEATS/forecast_6hour_regression_vis_combine_loss/1212_214019',
 '../exp/1212/IC_PN_BEATS/forecast_6hour_regression_vis_single_loss/1212_231549']

In [52]:
target_col_num = [10, 21, 32, 43, 54]
loss_list = ['Loss_1,2', 'Loss_2', 'Loss_1,3', 'Loss_3']
port_list = ['증산도', '불무기도', '장도', '목포', '여수항']

In [53]:
scaler = pickle.load(open('../src/data/SeaFog/graph_signal/scaler.pickle', 'rb'))
mean, std = scaler.x_shift, scaler.x_scale

mean = np.array(mean)
std = np.array(std)

mean = mean[target_col_num]
std = std[target_col_num]

In [54]:
loader = Temporal_Graph_Signal('std')
loader.path = '../src/data/SeaFog/graph_signal'
loader.preprocess_dataset()

train_dataset, _, test_dataset = loader.get_dataset(
            num_timesteps_in=input_length,
            num_timesteps_out=output_length,
            batch_size=1)

train_anomaly = []
for batch in train_dataset[0]:
    train_anomaly.append(batch.anomaly)
train_anomaly = torch.stack(train_anomaly, axis=0)

train_fog = sum(train_anomaly)

print(train_fog)
print(train_fog / train_anomaly.shape[0] * 100)

In [55]:
target_anomaly = []
for batch in test_dataset:
    target_anomaly.append(batch.anomaly)
target_anomaly = torch.stack(target_anomaly, axis=0)

In [56]:
total_result = {loss: {} for loss in loss_list}

for exp_dir in dirs:
    if exp_dir.split('/')[-2].split('_')[-3] == 'all':
        if exp_dir.split('/')[-2].split('_')[-2] == 'combine':
            name = 'Loss_1,2'
        else:
            name = "Loss_2"
    else:
        if exp_dir.split('/')[-2].split('_')[-2] == 'combine':
            name = 'Loss_1,3'
        else:
            name = "Loss_3"
            
    config, _, test_result = get_exp_result_files(exp_dir)
    
    if exp_dir.split('/')[-2].split('_')[-2] == 'combine':
        pred_anomaly = torch.sigmoid(torch.Tensor(test_result['anomaly_pred'])) > 0.5
        
    else:
        node_by_node_pred = []
        forecast = test_result['forecast'].squeeze()
        for node in range(5):
            port_forecast = forecast[:,node,:]

            inv_port_forecast = inv_std_scaler(port_forecast.reshape(-1), mean[node], std[node])
            inv_port_forecast = inv_port_forecast.reshape(forecast.shape[0], -1)

            inv_port_forecast = inv_port_forecast[:, -1]

            pred_anomaly = inv_port_forecast < 1000

            node_by_node_pred.append(torch.Tensor(pred_anomaly))

        pred_anomaly = torch.stack(node_by_node_pred, axis=0).T
        
    
    result = {'Acc': 0, 'Precision': 0, 'Recall': 0, 'F1': 0}

    _acc = accuracy_score(target_anomaly.reshape(-1), pred_anomaly.reshape(-1)) * 100
    _precision = precision_score(target_anomaly.reshape(-1), pred_anomaly.reshape(-1)) * 100
    _recall = recall_score(target_anomaly.reshape(-1), pred_anomaly.reshape(-1)) * 100
    _f1 = f1_score(target_anomaly.reshape(-1), pred_anomaly.reshape(-1)) * 100


    result['Acc'] = round(_acc, 2)
    result['Precision'] = round(_precision, 2)
    result['Recall'] = round(_recall, 2)
    result['F1'] = round(_f1, 2)
    
    total_result[name] = result

In [57]:
total_df = pd.DataFrame(total_result)
total_df.T

,Acc,Precision,Recall,F1
"Loss_1,2",97.62,32.84,65.96,43.85
Loss_2,99.11,87.72,42.55,57.31
"Loss_1,3",97.61,32.32,63.40,42.82
Loss_3,98.82,86.54,19.15,31.36


In [58]:
port_total_result = {loss: {} for loss in loss_list}

for exp_dir in dirs:
    if exp_dir.split('/')[-2].split('_')[-3] == 'all':
        if exp_dir.split('/')[-2].split('_')[-2] == 'combine':
            name = 'Loss_1,2'
        else:
            name = "Loss_2"
    else:
        if exp_dir.split('/')[-2].split('_')[-2] == 'combine':
            name = 'Loss_1,3'
        else:
            name = "Loss_3"
            
    config, _, test_result = get_exp_result_files(exp_dir)
    
    if exp_dir.split('/')[-2].split('_')[-2] == 'combine':
        pred_anomaly = torch.sigmoid(torch.Tensor(test_result['anomaly_pred'])) > 0.5
        
    else:
        node_by_node_pred = []
        forecast = test_result['forecast'].squeeze()
        
        for node in range(5):
            port_forecast = forecast[:,node,:]

            inv_port_forecast = inv_std_scaler(port_forecast.reshape(-1), mean[node], std[node])
            inv_port_forecast = inv_port_forecast.reshape(forecast.shape[0], -1)

            inv_port_forecast = inv_port_forecast[:, -1]

            pred_anomaly = inv_port_forecast < 1000

            node_by_node_pred.append(torch.Tensor(pred_anomaly))

        pred_anomaly = torch.stack(node_by_node_pred, axis=0).T
    
    
    port_result = {port: {} for port in port_list}    
    
    for idx, port in enumerate(port_list):
        _result = {'Acc': 0, 'Precision': 0, 'Recall': 0, 'F1': 0}

        _acc = accuracy_score(target_anomaly[:, idx], pred_anomaly[:, idx]) * 100
        _precision = precision_score(target_anomaly[:, idx], pred_anomaly[:, idx]) * 100
        _recall = recall_score(target_anomaly[:, idx], pred_anomaly[:, idx]) * 100
        _f1 = f1_score(target_anomaly[:, idx], pred_anomaly[:, idx]) * 100


        _result['Acc'] = round(_acc, 2)
        _result['Precision'] = round(_precision, 2)
        _result['Recall'] = round(_recall, 2)
        _result['F1'] = round(_f1, 2)

        port_result[port] = _result
        
    port_total_result[name] = port_result

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
loss_df = []
for loss in loss_list:
    loss_df.append(pd.DataFrame(port_total_result[loss]).T)

In [60]:
loss_df[1]

,Acc,Precision,Recall,F1
증산도,99.19,93.75,36.59,52.63
불무기도,98.68,86.67,50.65,63.93
장도,98.26,88.00,45.83,60.27
목포,99.43,66.67,10.00,17.39
여수항,99.97,0.00,0.00,0.00
